In [ ]:
!pip install sentencepiece
!pip install rouge

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import transformers
from numpy.random import randint

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Importing datasets

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#

# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1elNqcb5tW9snmf-xRfMK0AvLUfEcOhC1'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('news_summary.csv')
'''
file_id = '1rAKmHTbGPOeuREC7Olk5oNHjNrdFHKkz'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('news_summary_more.csv')
'''

"\nfile_id = '1rAKmHTbGPOeuREC7Olk5oNHjNrdFHKkz'\ndownloaded = drive.CreateFile({'id': file_id})\ndownloaded.GetContentFile('news_summary_more.csv')\n"

In [ ]:
data = pd.read_csv('news_summary.csv', encoding='latin-1') #check encodings types
#more_data = pd.read_csv('news_summary_more.csv', encoding='latin-1')

#Exploring the datasets

## news_summary dataset


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514 entries, 0 to 4513
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author     4514 non-null   object
 1   date       4514 non-null   object
 2   headlines  4514 non-null   object
 3   read_more  4514 non-null   object
 4   text       4514 non-null   object
 5   ctext      4396 non-null   object
dtypes: object(6)
memory usage: 211.7+ KB


In [ ]:
data.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [ ]:
duplicateRows1 = data[data.duplicated(subset=['ctext'])]
print('complete text duplicates')
print(duplicateRows1)

complete text duplicates
                  author  ...                                              ctext
42          Chhavi Tyagi  ...  The Daman and Diu administration on Wednesday ...
190         Chhavi Tyagi  ...  Charges and counter charges flew in the Lok Sa...
231   Niharika Prabhakar  ...                                                NaN
286        Saloni Tandon  ...                                                NaN
368         Chhavi Tyagi  ...  Bihar chief minister Nitish Kumar comfortably ...
...                  ...  ...                                                ...
4381        Chhavi Tyagi  ...  Rounding off a day of hectic electioneering in...
4423      Mansha Mahajan  ...                                                NaN
4454     Abhishek Bansal  ...                                                NaN
4500      Mansha Mahajan  ...                                                NaN
4508        Tarun Khanna  ...                                                NaN

[1

In [ ]:
duplicateRows2 = data[data.duplicated(subset=['text'])]
print('summary text duplicates',duplicateRows2)


summary text duplicates Empty DataFrame
Columns: [author, date, headlines, read_more, text, ctext]
Index: []


In [ ]:
duplicateRows3 = data[data.duplicated(subset=['headlines'])]
print('headlines duplicates',duplicateRows3)

headlines duplicates Empty DataFrame
Columns: [author, date, headlines, read_more, text, ctext]
Index: []


In [ ]:
selected_features = data[['headlines','text']]
selected_features.head()

,headlines,text
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Administration of Union Territory Daman an...
1,Malaika slams user who trolled her for 'divorc...,Malaika Arora slammed an Instagram user who tr...
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Maharashtra will train their staff t...


In [ ]:
selected_features.isnull().values.any()


False

In [ ]:
X = selected_features['text'].values
Y = selected_features['headlines'].values

numpy.ndarray

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)
X_train.shape, X_test.shape

((3611,), (903,))

In [ ]:
for i in range(10):
    idx = np.random.randint(0,3611)
    print(X_train[idx].split())

['Krishnamurthy,', 'a', 'resident', 'of', 'Erode', 'in', 'Tamil', 'Nadu,', 'has', 'filed', 'a', 'complaint', 'with', 'the', "state's", 'Chief', 'Secretary', 'claiming', 'that', 'he', 'is', 'the', 'only', 'son', 'of', 'Jayalalithaa', 'and', 'that', 'his', 'mother', 'was', 'killed.', '"She', '(Sasikala)', 'pushed', 'my', 'mother', 'down', 'the', 'stairs', 'on', '22nd', 'September', 'and', 'killed', 'her,"', 'Krishnamurthy', 'alleged.', 'Krishnamurthy', 'also', 'stated', 'that', 'he', 'is', 'the', 'rightful', 'heir', 'to', "Jayalalithaa's", 'properties.']
['More', 'than', '31', 'million', 'people,', 'that', 'is', 'one', 'person', 'every', 'second,', 'were', 'displaced', 'in', 'their', 'country', 'in', '2016', 'because', 'of', 'conflicts,', 'violence', 'and', 'natural', 'disasters,', 'a', 'Norwegian', 'Refugee', 'Council', 'report', 'has', 'revealed.', 'Adding', 'that', 'the', 'Democratic', 'Republic', 'of', 'the', 'Congo', 'was', 'the', 'worst', 'affected', 'country,', 'the', 'report', 's

# Abstractive Models

##GOOGLE's PEGASUS

In [ ]:
indx = randint(len(X_train),size = 10)

In [ ]:
peg_model = transformers.pipeline('summarization','google/pegasus-xsum')

In [ ]:
peg_sents = [""]*len(indx)
j = 0
for i in indx:
  peg_sents[j] = peg_model(X_train[i])[0]['summary_text']
  print('The article is: {}\nThe headline is: {}\nThe generated summary is:'.format(X_train[i], y_train[i]),peg_sents[j])
  j+=1

Your max_length is set to 64, but you input_length is only 63. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


The article is: At least 19 people were killed in different incidents of lightning strikes in Uttar Pradesh and Bihar. The most severely hit was Bihar, where 16 people died and two were injured in seven districts due to the strikes. Meanwhile, the death toll in Assam rose to 26 after one more life was claimed by the floods.
The headline is: Lightning kills 19 in UP, Bihar; 26 killed in Assam floods
The generated summary is: At least 39 people have been killed in separate incidents of lightning strikes in India's Bihar and Uttar Pradesh states in the past 24 hours.
The article is: A Kotak Mahindra Bank branch manager arrested on Wednesday, Ashish Kumar, took ?1.5 crore to convert ?35 crore demonetised currency of Delhi-based lawyer Rohit Tandon, said the Enforcement Directorate. The bank has suspended the services of the manager. Earlier in December, two people were arrested for opening fake accounts in the branch with unaccounted deposits totalling ?34 crore.
The headline is: Kotak Mah

Your max_length is set to 64, but you input_length is only 62. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


The article is: As per a survey conducted by the World Water Council (WWC), 71% Indians believe the standard of drinking water in India has improved in the last five years. "I think Mr Modi is very much concerned with water and sanitation issues. He has even appointed a special secretary to deal with the Ganga river cleanup," said WWC President Benedito Braga.n
The headline is: Drinking water standards have improved in India: WWC
The generated summary is: India's Prime Minister Narendra Modi has been praised for his focus on water and sanitation issues.
The article is: The Bombay High Court on Tuesday slammed over 4,500 resident doctors in Maharashtra who had gone on mass leave since March 20 and asked them to join work or face action by the management. The HC was hearing a PIL filed against the strike wherein the advocate representing the petitioners claimed 58 people have died in Maharashtra since Monday.
The headline is: Join work or face action: Bombay HC to protesting doctors
The 

##facebook/bart-large-cnn

In [ ]:
smr_bart = transformers.pipeline(task="summarization", model="facebook/bart-large-cnn")

In [ ]:
smr_bart_sents = [""]*len(indx)
j = 0
for i in indx:
    smr_bart_sents[j] = smr_bart(X_train[i],max_length=20)[0]['summary_text']
    print('The article is: {}\nThe headline is: {}\nThe generated summary is:'.format(X_train[i], y_train[i]),smr_bart_sents[j])
    j+=1

The article is: At least 19 people were killed in different incidents of lightning strikes in Uttar Pradesh and Bihar. The most severely hit was Bihar, where 16 people died and two were injured in seven districts due to the strikes. Meanwhile, the death toll in Assam rose to 26 after one more life was claimed by the floods.
The headline is: Lightning kills 19 in UP, Bihar; 26 killed in Assam floods
The generated summary is: At least 19 people were killed in different incidents of lightning strikes in Uttar Pradesh and Bihar
The article is: A Kotak Mahindra Bank branch manager arrested on Wednesday, Ashish Kumar, took ?1.5 crore to convert ?35 crore demonetised currency of Delhi-based lawyer Rohit Tandon, said the Enforcement Directorate. The bank has suspended the services of the manager. Earlier in December, two people were arrested for opening fake accounts in the branch with unaccounted deposits totalling ?34 crore.
The headline is: Kotak Mahindra Bank manager took ?1.5 cr to conver

In [ ]:
y_train[indx[4]]

##google/roberta2roberta_L-24_bbc

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/roberta2roberta_L-24_bbc")
model = AutoModelForSeq2SeqLM.from_pretrained("google/roberta2roberta_L-24_bbc")
sents = []
j = 0
for i in indx:
  input_ids = tokenizer(X_train[i], return_tensors="pt").input_ids
  output_ids = model.generate(input_ids)[0]
  sents.append(tokenizer.decode(output_ids, skip_special_tokens=True))
  print('The article is: {}\nThe headline is: {}\nThe generated summary is:'.format(X_train[i], y_train[i]),sents[j])
  j+=1


The article is: At least 19 people were killed in different incidents of lightning strikes in Uttar Pradesh and Bihar. The most severely hit was Bihar, where 16 people died and two were injured in seven districts due to the strikes. Meanwhile, the death toll in Assam rose to 26 after one more life was claimed by the floods.
The headline is: Lightning kills 19 in UP, Bihar; 26 killed in Assam floods
The generated summary is: More than 100 people have been killed and more than 100 others injured in floods and landslides in India and the northern states of Assam and Assam.
The article is: A Kotak Mahindra Bank branch manager arrested on Wednesday, Ashish Kumar, took ?1.5 crore to convert ?35 crore demonetised currency of Delhi-based lawyer Rohit Tandon, said the Enforcement Directorate. The bank has suspended the services of the manager. Earlier in December, two people were arrested for opening fake accounts in the branch with unaccounted deposits totalling ?34 crore.
The headline is: Kot

In [ ]:
print(X_train[indx[9]])

The new Nokia 3310 has a 2.4-inch colour screen unlike the original phone, which came with an 84x84 black and white display. The 2017 Nokia 3310's 1200mAh battery life is 10 times more than the original. The new model, which is slimmer and lighter than the original, has an updated version of the 'Snake' game, which was made by Gameloft.


##Evaluation

In [ ]:
from rouge import Rouge

In [ ]:
print('google/pegasus-xsum rouge-1 score')
rouge_metric = Rouge()
score = rouge_metric.get_scores(list(y_train[indx]), peg_sents, avg=True)
rouge1 = score.get('rouge-1')
print(rouge1)

google/pegasus-xsum rouge-1 score
{'f': 0.15234493824976872, 'p': 0.23641774891774894, 'r': 0.12066067483714545}


In [ ]:
print('facebook/bart-large-cnn rouge-1 score')
rouge_metric = Rouge()
score = rouge_metric.get_scores(list(y_train[indx]), smr_bart_sents , avg=True)
rouge1 = score.get('rouge-1')
print(rouge1)

facebook/bart-large-cnn rouge-1 score
{'f': 0.3421554529896391, 'p': 0.38953463203463207, 'r': 0.31358477307006716}


In [ ]:
print('google/roberta2roberta_L-24_bbc rouge-1 score')
rouge_metric = Rouge()
score = rouge_metric.get_scores(list(y_train[indx]), sents, avg=True)
rouge1 = score.get('rouge-1')
print(rouge1)

google/roberta2roberta_L-24_bbc rouge-1 score
{'f': 0.1400761432407765, 'p': 0.2193217893217893, 'r': 0.10463552823066216}
